In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Exploration
- Let's extract the data first and concat the two json file into single csv file. Save the csv file
- Data exploration afterwards

In [2]:
# df = pd.read_json('gumtree.json')
# df2 = pd.read_json('remaininggumtree.json')
# data = pd.concat([df,df2])
# #Saving it as a final_data.csv
# data.to_csv('final_data.csv',index=False)

In [3]:
dataset = pd.read_csv('final_data.csv')

In [4]:
#Let's check out the coloumn names
dataset.columns #All the coloumns here are important so we won't be deleting it. 

Index(['Year', 'Body type', 'Mileage', 'Transmission', 'Brochure Engine size',
       'Fuel type', 'Urban mpg', 'Extra Urban mpg', 'Fuel Consumption',
       'Insurance Group', 'CO2 emissions', 'Euro Emissions', 'Engine Power',
       'Engine size', 'Acceleration (0-62mph)', 'Top Speed', 'Seats', 'Doors',
       'Colour', 'Fuel Capacity', 'Luggage Capacity (seats up)', 'Price',
       'Name', 'Location'],
      dtype='object')

In [5]:
#However some of the coloumns needs some renaming to be done 
#The reason we are doing this is because we want
dataset.rename({'Acceleration (0-62mph)':'Acceleration(0-62mph)','Luggage Capacity (seats up)':'Luggage Capacity(seats_up)'},inplace=True)

In [17]:
#Checking if there is any duplicate values in the dataset
#Let's find out duplicate values
dataset.duplicated().sum() #There are alltogether 300 values that are duplicate
#Let's see what those duplicate values are
duplicates = dataset[dataset.duplicated()]

In [21]:
duplicates.to_csv('duplicates.csv',index=False) #Exporting to csv to further analyse the data properly.(Not necessary)

In [24]:
#Let's remove the duplciates from the main dataset
final_dataset = dataset.drop_duplicates()

In [26]:
final_dataset #We sucessfully removed all the 300 duplicates from the data. 
#Before the data dimanesion was 1078 rows × 24 columns and now after removing 300 duplicate values, it is 778 rows × 24 columns

,Year,Body type,Mileage,Transmission,Brochure Engine size,Fuel type,Urban mpg,Extra Urban mpg,Fuel Consumption,Insurance Group,...,Acceleration (0-62mph),Top Speed,Seats,Doors,Colour,Fuel Capacity,Luggage Capacity (seats up),Price,Name,Location
0,2017,Other,"31,000miles",Manual,1.0L,Petrol,42.8mpg,60.1mpg,52.3mpg,13E,...,11.8seconds,115mph,5.0,5.0,Blue,52litres,310litres,"£9,495.00",2017 Ford Ecosport 1.0 T EcoBoost Titanium S 5...,"St Georges, Shropshire"
1,2008,Hatchback,"116,264miles",Manual,1.3L,Diesel,49.6mpg,74.3mpg,62.8mpg,5E,...,13.6seconds,101mph,5.0,5.0,Blue,45litres,285litres,"£1,195.00","Vauxhall, CORSA, Hatchback, 2008, Manual, 1248...","Durham, County Durham"
2,2009,Hatchback,"24,000miles",Manual,1.2L,Petrol,38.2mpg,55.4mpg,47.9mpg,11E,...,13.5seconds,104mph,5.0,3.0,Other,46litres,251litres,"£2,195.00","Nissan, MICRA, Acenta Plus, Hatchback, 2009, M...","Sunderland, Tyne and Wear"
3,2006,Hatchback,"112,349miles",Manual,2.0L,Petrol,26.2mpg,45.6mpg,35.8mpg,28E,...,7.1seconds,146mph,5.0,3.0,Red,55litres,350litres,"£4,495.00",Audi A3 2.0T FSI Quattro</h1,"Moira, County Armagh"
4,2015,Hatchback,"42,187miles",Automatic,1.4L,Petrol,36.7mpg,57.6mpg,47.9mpg,6E,...,13.9seconds,106mph,5.0,5.0,Other,45litres,285litres,"£6,980.00",2015 Vauxhall Corsa SE AUTO Hatchback Petrol A...,"Chester, Cheshire"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,2015,Hatchback,"40,000miles",Manual,1.6L,Diesel,65.7mpg,91.1mpg,80.7mpg,19E,...,9.5seconds,124mph,5.0,5.0,Blue,45litres,270litres,"£11,495.00",2015 15 AUDI A1 1.6 SPORTBACK TDI S LINE 5D 11...,"Pontarddulais, Swansea"
1074,2012,Hatchback,"54,000miles",Manual,1.0L,Petrol,50.4mpg,67.3mpg,60.1mpg,7E,...,14.7seconds,99mph,5.0,5.0,Red,45litres,178litres,"£3,495.00",2012 62 SUZUKI SPLASH 1.0 SZ2 5D 68 BHP</h1,"Pontarddulais, Swansea"
1075,2019,Hatchback,"2,373miles",Automatic,1.5L,Diesel,62.8mpg,70.6mpg,67.3mpg,15E,...,11.6seconds,119mph,5.0,5.0,Grey,53litres,514litres,"£17,711.00",2019 Vauxhall Grandland X Grandland X 1.5 Turb...,"Telford, Shropshire"
1076,2016,Hatchback,"47,000miles",Manual,1.2L,Petrol,54.3mpg,76.3mpg,65.7mpg,15E,...,9.6seconds,118mph,5.0,3.0,Black,50litres,285litres,"£6,495.00",2016 Peugeot 208 1.2 PureTech GT Line (s/s) 3d...,"Thrapston, Northamptonshire"


In [30]:
#Let's now count how many missing values are in the dataset
#Since the missing values are just blank in the dataset, let's fill those values with nan 
final_dataset = final_dataset.replace(r'^\s*$',np.nan,regex = True) #Using regex to replace all the white spaces with NaN

In [32]:
final_dataset.isna().sum()

Year                            0
Body type                       0
Mileage                         0
Transmission                    0
Brochure Engine size            4
Fuel type                       0
Urban mpg                      51
Extra Urban mpg                50
Fuel Consumption                8
Insurance Group                16
CO2 emissions                   1
Euro Emissions                  4
Engine Power                    1
Engine size                     0
Acceleration (0-62mph)         13
Top Speed                       1
Seats                           1
Doors                           3
Colour                          0
Fuel Capacity                  39
Luggage Capacity (seats up)    41
Price                           0
Name                            0
Location                        0
dtype: int64